In [155]:
# make sure we have the most current version of github repo
%cd Satellite_ComputerVision
!git pull
%cd ..

/mnt/batch/tasks/shared/LS_root/mounts/clusters/wetlandsbasiccpu/code/Users/mevans/Satellite_ComputerVision
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 20 (delta 16), reused 19 (delta 15), pack-reused 0
Unpacking objects: 100% (20/20), 5.52 KiB | 2.00 KiB/s, done.
From https://github.com/mjevans26/Satellite_ComputerVision
   164ff20..ce176df  master     -> origin/master
Updating 164ff20..ce176df
Fast-forward
 .spyproject/workspace.ini |   2 +-
 azure/retrain_wetland.py  | 232 ++++++++++++++++++++++++++++++++++++++++++++++
 azure/train_acd.py        |   4 +-
 azure/train_wetland.py    |  48 +++++-----
 utils/model_tools.py      |  18 ++--
 utils/prediction_tools.py |   4 +-
 6 files changed, 276 insertions(+), 32 deletions(-)
 create mode 100644 azure/retrain_wetland.py
/mnt/batch/tasks/shared/LS_root/mounts/clusters/wetlandsbasiccpu/code/Users/mevans


In [1]:
from azureml.tensorboard import Tensorboard
from azureml.core import Experiment, Environment, Workspace, Datastore, Dataset, Model, ScriptRunConfig
import os
import glob
# get the current workspace
ws = Workspace.from_config()

In [ ]:
# access our registered data share containing image data in this workspace
datastore = Datastore.get(workspace = ws, datastore_name = 'wetlanddatastore')

train_path = (datastore, 'summerReplicated/training/')
eval_path = (datastore, 'summerReplicated/eval/')

test_path = (datastore, 'summerReplicated/predict/test_aoi_David1_DE')

train_dataset = Dataset.File.from_files(path = [train_path])
eval_dataset = Dataset.File.from_files(path = [eval_path])
test_dataset = Dataset.File.from_files(path = [test_path])

In [4]:
# FInd the run corresponding to the model we want to register
run_id = 'wetland-unet_1624570645_35af4ca5'

run = ws.get_run(run_id)

run.tags

{'splits': 'None',
 'one_hot': 'geomorphons',
 'model': 'Unet',
 'normalization': 'pixel',
 'features': 'full',
 'epochs': '100-300'}

In [5]:
model_name = 'wetland-unet-full'

In [195]:
# if we haven't already registered our model
model = run.register_model(model_name= model_name,
                           tags=run.tags,
                           description = 'UNET model delineating wetlands using Sentinel-2, NAIP data from DE, MN, NY.',
                           model_path='outputs/',
                           model_framework = 'Tensorflow',
                           model_framework_version= '2.0',
                           datasets = [('training', train_dataset), ('evaluation', eval_dataset), ('testing', test_dataset)])
print(model.name, model.id, model.version, sep='\t')

wetland-unet-basic	wetland-unet-basic:5	5


In [6]:
model_id = model_name
model = Model(ws, model_id)
model.version
model.get_model_path(model_id, _workspace = ws)

# model_path = model.get_model_path(model_id, _workspace = ws)
# model_path

Cannot find Saved Dataset with ID f5293f38-4bc7-46ab-bbd1-0ef61f4293ba due to the following exception.
Engine process terminated. This is most likely due to system running out of memory. Please retry with increased memory. |session_id=981730c8-31eb-4119-a3ee-fab8b1b7bba6
Unable to retrieve Dataset with ID f5293f38-4bc7-46ab-bbd1-0ef61f4293ba due to the following exception.
__enter__
Cannot find Saved Dataset with ID 7856ae8c-8ca5-4834-b856-c767ff79ae7d due to the following exception.
Engine process terminated. This is most likely due to system running out of memory. Please retry with increased memory. |session_id=981730c8-31eb-4119-a3ee-fab8b1b7bba6
Unable to retrieve Dataset with ID 7856ae8c-8ca5-4834-b856-c767ff79ae7d due to the following exception.
__enter__
Cannot find Saved Dataset with ID 3a762777-f939-4376-a66f-72a175b571ed due to the following exception.
Engine process terminated. This is most likely due to system running out of memory. Please retry with increased memory. |sess

PermissionError: [Errno 13] Permission denied: 'azureml-models'

In [7]:
Model.get_model_path(model_id, _workspace = ws)

Cannot find Saved Dataset with ID f5293f38-4bc7-46ab-bbd1-0ef61f4293ba due to the following exception.
Engine process terminated. This is most likely due to system running out of memory. Please retry with increased memory. |session_id=981730c8-31eb-4119-a3ee-fab8b1b7bba6
Unable to retrieve Dataset with ID f5293f38-4bc7-46ab-bbd1-0ef61f4293ba due to the following exception.
__enter__
Cannot find Saved Dataset with ID 7856ae8c-8ca5-4834-b856-c767ff79ae7d due to the following exception.
Engine process terminated. This is most likely due to system running out of memory. Please retry with increased memory. |session_id=981730c8-31eb-4119-a3ee-fab8b1b7bba6
Unable to retrieve Dataset with ID 7856ae8c-8ca5-4834-b856-c767ff79ae7d due to the following exception.
__enter__
Cannot find Saved Dataset with ID 3a762777-f939-4376-a66f-72a175b571ed due to the following exception.
Engine process terminated. This is most likely due to system running out of memory. Please retry with increased memory. |sess

PermissionError: [Errno 13] Permission denied: 'azureml-models'

In [8]:
# use the azure folder as our script folder
source = 'Satellite_ComputerVision'
util_folder = 'utils'
script_folder = 'azure'
script_file = 'retrain_wetland_{}.py'

In [9]:
import shutil
shutil.copytree(src = f'./azureml-models/{model_name}', dst = f'./azure/models/{model_name}')

'./azure/models/wetland-unet-basic'

In [9]:
# get our environment
envs = Environment.list(workspace = ws)
env = envs.get('wetland-training')

In [10]:
# define the compute target
ws.compute_targets
wetlandGPU = ws.compute_targets['wetlandsGPU']
wetlandDSVM = ws.compute_targets['wetland-dsvm']
wetlandDSVM2 = ws.compute_targets['wetland-dsvm2']

In [11]:
experiment_name = 'wetland-unet-les'
exp = Experiment(workspace = ws, name = experiment_name)

In [207]:
# define the command line arguments to our training sript
args = [
    '--train_data', train_dataset.as_mount(),
    '--eval_data', eval_dataset.as_mount(),
    '--test_data', test_dataset.as_mount(),
    '--model_id', model_id,
    '--weight', 1, 
    '--bias', 0,
    '-lr', 0.0005,
    '--epochs', 100,
    '--batch', 16,
    '--size', 7207, #14225,
    '--kernel_size', 256,
    '--response', 'wetland']

In [210]:
#  run the training job
features = 'basic'
src = ScriptRunConfig(source_directory=script_folder,
                      script=script_file.format(features), 
                      arguments=args,
                      compute_target=wetlandGPU,
                      environment=env)
run = exp.submit(config=src, tags = dict({'splits':'None', 'one_hot':'None', 'model':'Unet', 'normalization':'pixel', 'features':f'{features}', 'epochs':'160-300'}))
run

Experiment,Id,Type,Status,Details Page,Docs Page
wetland-unet,wetland-unet_1624846917_1dc76912,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [181]:
#  run the training job
features = 'wgeomorphon'
src = ScriptRunConfig(source_directory=script_folder,
                      script=script_file.format(features), 
                      arguments=args,
                      compute_target= wetlandDSVM2,
                      environment=env)
run = exp.submit(config=src, tags = dict({'splits':'None', 'one_hot':'geomorphons', 'model':'Unet', 'normalization':'pixel', 'features':f'{features}', 'epochs':'100-300'}))
run

Experiment,Id,Type,Status,Details Page,Docs Page
wetland-unet,wetland-unet_1624591441_c5d8f43a,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation
